In [1]:
import os
import sys
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.relativedelta import relativedelta
import pickle
from ast import literal_eval

pd.options.display.max_columns=100
pd.options.display.max_rows=1000
pd.options.display.max_colwidth = 500

# Load data, prep for merging

In [2]:
# load Andriy's latest WB dataset (as of 10/29/17)
with open('./datasets/final_wb_array copy.txt') as f: # removed "var geo =" from original final_wb_array.txt
    wb = [literal_eval(line) for line in f]
wb=wb[0]
wb = pd.DataFrame.from_records(wb[:-1], columns=wb[-1])
wb.tail(2)

,long,lat,code2,country,ARI treatment (% of children under 5 taken to a health provider)' 2007,ARI treatment (% of children under 5 taken to a health provider)' 2008,ARI treatment (% of children under 5 taken to a health provider)' 2009,ARI treatment (% of children under 5 taken to a health provider)' 2010,ARI treatment (% of children under 5 taken to a health provider)' 2011,ARI treatment (% of children under 5 taken to a health provider)' 2012,ARI treatment (% of children under 5 taken to a health provider)' 2013,ARI treatment (% of children under 5 taken to a health provider)' 2014,ARI treatment (% of children under 5 taken to a health provider)' 2015,Birth rate crude (per 1000 people)' 2007,Birth rate crude (per 1000 people)' 2008,Birth rate crude (per 1000 people)' 2009,Birth rate crude (per 1000 people)' 2010,Birth rate crude (per 1000 people)' 2011,Birth rate crude (per 1000 people)' 2012,Birth rate crude (per 1000 people)' 2013,Birth rate crude (per 1000 people)' 2014,Birth rate crude (per 1000 people)' 2015,Births attended by skilled health staff (% of total)' 2007,Births attended by skilled health staff (% of total)' 2008,Births attended by skilled health staff (% of total)' 2009,Births attended by skilled health staff (% of total)' 2010,Births attended by skilled health staff (% of total)' 2011,Births attended by skilled health staff (% of total)' 2012,Births attended by skilled health staff (% of total)' 2013,Births attended by skilled health staff (% of total)' 2014,Births attended by skilled health staff (% of total)' 2015,Depth of the food deficit (kilocalories per person per day)' 2007,Depth of the food deficit (kilocalories per person per day)' 2008,Depth of the food deficit (kilocalories per person per day)' 2009,Depth of the food deficit (kilocalories per person per day)' 2010,Depth of the food deficit (kilocalories per person per day)' 2011,Depth of the food deficit (kilocalories per person per day)' 2012,Depth of the food deficit (kilocalories per person per day)' 2013,Depth of the food deficit (kilocalories per person per day)' 2014,Depth of the food deficit (kilocalories per person per day)' 2015,External resources for health (% of total expenditure on health)' 2007,External resources for health (% of total expenditure on health)' 2008,External resources for health (% of total expenditure on health)' 2009,External resources for health (% of total expenditure on health)' 2010,External resources for health (% of total expenditure on health)' 2011,External resources for health (% of total expenditure on health)' 2012,External resources for health (% of total expenditure on health)' 2013,External resources for health (% of total expenditure on health)' 2014,External resources for health (% of total expenditure on health)' 2015,Fertility rate total (births per woman)' 2007,...,Prevalence of underweight weight for age female (% of children under 5)' 2011,Prevalence of underweight weight for age female (% of children under 5)' 2012,Prevalence of underweight weight for age female (% of children under 5)' 2013,Prevalence of underweight weight for age female (% of children under 5)' 2014,Prevalence of underweight weight for age female (% of children under 5)' 2015,Prevalence of underweight weight for age male (% of children under 5)' 2007,Prevalence of underweight weight for age male (% of children under 5)' 2008,Prevalence of underweight weight for age male (% of children under 5)' 2009,Prevalence of underweight weight for age male (% of children under 5)' 2010,Prevalence of underweight weight for age male (% of children under 5)' 2011,Prevalence of underweight weight for age male (% of children under 5)' 2012,Prevalence of underweight weight for age male (% of children under 5)' 2013,Prevalence of underweight weight for age male (% of children under 5)' 2014,Prevalence of underweight weight for age male (% of children under 5)' 2015,Survival to age 65 female (% of cohort)' 2007,Survival to age 65 female (% of co

In [3]:
# load Andriy's previous dataset - need countries "code" to merge with Matt's data
wb_old = pd.read_csv('datasets/health_data_wb.csv')
wb_old.head(2)

,country,code,indicator_name,year,indicator,code2
0,Afghanistan,AFG,ARI treatment (% of children under 5 taken to a health provider),2007,0.00,AF
1,Afghanistan,AFG,"Birth rate, crude (per 1,000 people)",2007,42.86,AF


In [4]:
# replace 'code2' with 'code' 
code_mapping = dict(wb_old[['code2', 'code']].values) # separate step
wb['code2'] = wb['code2'].replace(code_mapping)
wb.rename(columns={'code2':'code'}, inplace=True)
wb.head(2)

,long,lat,code,country,ARI treatment (% of children under 5 taken to a health provider)' 2007,ARI treatment (% of children under 5 taken to a health provider)' 2008,ARI treatment (% of children under 5 taken to a health provider)' 2009,ARI treatment (% of children under 5 taken to a health provider)' 2010,ARI treatment (% of children under 5 taken to a health provider)' 2011,ARI treatment (% of children under 5 taken to a health provider)' 2012,ARI treatment (% of children under 5 taken to a health provider)' 2013,ARI treatment (% of children under 5 taken to a health provider)' 2014,ARI treatment (% of children under 5 taken to a health provider)' 2015,Birth rate crude (per 1000 people)' 2007,Birth rate crude (per 1000 people)' 2008,Birth rate crude (per 1000 people)' 2009,Birth rate crude (per 1000 people)' 2010,Birth rate crude (per 1000 people)' 2011,Birth rate crude (per 1000 people)' 2012,Birth rate crude (per 1000 people)' 2013,Birth rate crude (per 1000 people)' 2014,Birth rate crude (per 1000 people)' 2015,Births attended by skilled health staff (% of total)' 2007,Births attended by skilled health staff (% of total)' 2008,Births attended by skilled health staff (% of total)' 2009,Births attended by skilled health staff (% of total)' 2010,Births attended by skilled health staff (% of total)' 2011,Births attended by skilled health staff (% of total)' 2012,Births attended by skilled health staff (% of total)' 2013,Births attended by skilled health staff (% of total)' 2014,Births attended by skilled health staff (% of total)' 2015,Depth of the food deficit (kilocalories per person per day)' 2007,Depth of the food deficit (kilocalories per person per day)' 2008,Depth of the food deficit (kilocalories per person per day)' 2009,Depth of the food deficit (kilocalories per person per day)' 2010,Depth of the food deficit (kilocalories per person per day)' 2011,Depth of the food deficit (kilocalories per person per day)' 2012,Depth of the food deficit (kilocalories per person per day)' 2013,Depth of the food deficit (kilocalories per person per day)' 2014,Depth of the food deficit (kilocalories per person per day)' 2015,External resources for health (% of total expenditure on health)' 2007,External resources for health (% of total expenditure on health)' 2008,External resources for health (% of total expenditure on health)' 2009,External resources for health (% of total expenditure on health)' 2010,External resources for health (% of total expenditure on health)' 2011,External resources for health (% of total expenditure on health)' 2012,External resources for health (% of total expenditure on health)' 2013,External resources for health (% of total expenditure on health)' 2014,External resources for health (% of total expenditure on health)' 2015,Fertility rate total (births per woman)' 2007,...,Prevalence of underweight weight for age female (% of children under 5)' 2011,Prevalence of underweight weight for age female (% of children under 5)' 2012,Prevalence of underweight weight for age female (% of children under 5)' 2013,Prevalence of underweight weight for age female (% of children under 5)' 2014,Prevalence of underweight weight for age female (% of children under 5)' 2015,Prevalence of underweight weight for age male (% of children under 5)' 2007,Prevalence of underweight weight for age male (% of children under 5)' 2008,Prevalence of underweight weight for age male (% of children under 5)' 2009,Prevalence of underweight weight for age male (% of children under 5)' 2010,Prevalence of underweight weight for age male (% of children under 5)' 2011,Prevalence of underweight weight for age male (% of children under 5)' 2012,Prevalence of underweight weight for age male (% of children under 5)' 2013,Prevalence of underweight weight for age male (% of children under 5)' 2014,Prevalence of underweight weight for age male (% of children under 5)' 2015,Survival to age 65 female (% of cohort)' 2007,Survival to age 65 female (% of coh

In [5]:
# Matt's WHO mortality data
with open('WHO_mortality_data/mort_by_cause_per_capita_allages.pkl', 'rb') as f:
    who = pickle.load(f)  
who=who[who['sex']=='BTSX'] # subset only both-sex data for simplicity
who.drop(['sex','cause2015'], axis=1, inplace=True)
who.rename(columns={'country_name':'country', 'iso3':'code'}, inplace=True)
who.head(2)

,code,country,causename,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,AFG,Afghanistan,All Causes,1242.960127,1230.340822,1131.914864,1062.525087,1033.117750,1011.352244,1010.784279,988.040863,958.217465,938.247619,909.354213,882.969523,854.130237,811.421535,801.330483,770.549236
3,AFG,Afghanistan,"Communicable, maternal, perinatal and nutritional conditions",740.960043,733.359046,661.115117,593.481844,558.549633,535.716020,518.459724,487.932345,472.701997,461.366611,432.494270,410.523398,384.240053,345.329560,322.453669,303.872899


In [6]:
# countries' Region name and income group, for merging
countries_class=pd.read_excel('datasets/CLASS.xls', header=3, index_col='x').drop(['x.1','X'],axis=1).drop('x',axis=0)
countries_class.rename(columns={'Code':'code'}, inplace=True)
countries_class.head(2)

,Economy,code,Region,Income group,Lending category,Other
x,,,,,,
1,Afghanistan,AFG,South Asia,Low income,IDA,HIPC
2,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN


# Countries in WB vs WHO data

In [7]:
# number of countries in WB and WHO data 
len(set(who.code) &set(wb.code))

157

In [8]:
# countries in WB that are not in WHO
print(len(set(who.code)-set(wb.code)), "countries that in WB data that are not in WHO data")
set(who.country[who.code.isin(set(who.code)-set(wb.code))])

26 countries that in WB data that are not in WHO data


{'Bahamas',
 'Brunei_Darussalam',
 'Cabo_Verde',
 'Congo',
 "Côte_d'Ivoire",
 "Democratic_People's_Republic_of_Korea",
 'Democratic_Republic_of_the_Congo',
 'Egypt',
 'Gambia',
 'Iran_(Islamic_Republic_of)',
 'Kyrgyzstan',
 "Lao_People's_Democratic_Republic",
 'Micronesia_(Federated_States_of)',
 'Myanmar',
 'Namibia',
 'Republic_of_Korea',
 'Russian_Federation',
 'Saint_Lucia',
 'Saint_Vincent_and_the_Grenadines',
 'Sao_Tome_and_Principe',
 'Slovakia',
 'South_Sudan',
 'Syrian_Arab_Republic',
 'The_former_Yugoslav_Republic_of_Macedonia',
 'Venezuela_(Bolivarian_Republic_of)',
 'Yemen'}

In [9]:
# countries in WHO that are not in WB
print(len(set(wb.code)-set(who.code)), "countries that in WHO data that are not in WB data")
set(wb.country[wb.code.isin(set(wb.code)-set(who.code))])

23 countries that in WHO data that are not in WB data


{'American Samoa',
 'Andorra',
 'Aruba',
 'Bermuda',
 'British Virgin Islands',
 'Cayman Islands',
 'Dominica',
 'Faroe Islands',
 'French Polynesia',
 'Greenland',
 'Guam',
 'Kosovo',
 'Liechtenstein',
 'Marshall Islands',
 'Monaco',
 'Nauru',
 'New Caledonia',
 'Northern Mariana Islands',
 'Palau',
 'Puerto Rico',
 'San Marino',
 'Turks and Caicos Islands',
 'Tuvalu'}

# merge WB and WHO data

In [10]:
year_lst=['2007', '2008', '2009', '2010','2011', '2012', '2013', '2014', '2015']
wb_who_merge = wb.copy()
for year in year_lst:  
    who_pivot = who.pivot_table(index=['code'], columns='causename', values=year)
    who_pivot.columns = [str(col) + "' " + year for col in who_pivot.columns]
    who_pivot.reset_index(inplace=True)
    wb_who_merge = wb_who_merge.merge(who_pivot, on='code', how='inner')
wb_who_merge.head()

,long,lat,code,country,ARI treatment (% of children under 5 taken to a health provider)' 2007,ARI treatment (% of children under 5 taken to a health provider)' 2008,ARI treatment (% of children under 5 taken to a health provider)' 2009,ARI treatment (% of children under 5 taken to a health provider)' 2010,ARI treatment (% of children under 5 taken to a health provider)' 2011,ARI treatment (% of children under 5 taken to a health provider)' 2012,ARI treatment (% of children under 5 taken to a health provider)' 2013,ARI treatment (% of children under 5 taken to a health provider)' 2014,ARI treatment (% of children under 5 taken to a health provider)' 2015,Birth rate crude (per 1000 people)' 2007,Birth rate crude (per 1000 people)' 2008,Birth rate crude (per 1000 people)' 2009,Birth rate crude (per 1000 people)' 2010,Birth rate crude (per 1000 people)' 2011,Birth rate crude (per 1000 people)' 2012,Birth rate crude (per 1000 people)' 2013,Birth rate crude (per 1000 people)' 2014,Birth rate crude (per 1000 people)' 2015,Births attended by skilled health staff (% of total)' 2007,Births attended by skilled health staff (% of total)' 2008,Births attended by skilled health staff (% of total)' 2009,Births attended by skilled health staff (% of total)' 2010,Births attended by skilled health staff (% of total)' 2011,Births attended by skilled health staff (% of total)' 2012,Births attended by skilled health staff (% of total)' 2013,Births attended by skilled health staff (% of total)' 2014,Births attended by skilled health staff (% of total)' 2015,Depth of the food deficit (kilocalories per person per day)' 2007,Depth of the food deficit (kilocalories per person per day)' 2008,Depth of the food deficit (kilocalories per person per day)' 2009,Depth of the food deficit (kilocalories per person per day)' 2010,Depth of the food deficit (kilocalories per person per day)' 2011,Depth of the food deficit (kilocalories per person per day)' 2012,Depth of the food deficit (kilocalories per person per day)' 2013,Depth of the food deficit (kilocalories per person per day)' 2014,Depth of the food deficit (kilocalories per person per day)' 2015,External resources for health (% of total expenditure on health)' 2007,External resources for health (% of total expenditure on health)' 2008,External resources for health (% of total expenditure on health)' 2009,External resources for health (% of total expenditure on health)' 2010,External resources for health (% of total expenditure on health)' 2011,External resources for health (% of total expenditure on health)' 2012,External resources for health (% of total expenditure on health)' 2013,External resources for health (% of total expenditure on health)' 2014,External resources for health (% of total expenditure on health)' 2015,Fertility rate total (births per woman)' 2007,...,Other sense organ disorders' 2015,Other unintentional injuries' 2015,Other urinary diseases' 2015,Other vision loss' 2015,Otitis media' 2015,Ovary cancer' 2015,Pancreas cancer' 2015,Pancreatitis' 2015,Paralytic ileus and intestinal obstruction' 2015,Parasitic and vector diseases' 2015,Parkinson disease' 2015,Peptic ulcer disease' 2015,Periodontal disease' 2015,Poisonings' 2015,Preterm birth complications' 2015,Prostate cancer' 2015,Protein-energy malnutrition' 2015,Rabies' 2015,Respiratory diseases' 2015,Respiratory infections ' 2015,Rheumatic heart disease' 2015,Rheumatoid arthritis' 2015,Road injury' 2015,STDs excluding HIV' 2015,Schistosomiasis' 2015,Schizophrenia' 2015,Self-harm' 2015,Sense organ diseases' 2015,Sickle cell disorders and trait' 2015,Skin diseases' 2015,Stomach cancer' 2015,Stroke' 2015,Sudden infant death syndrome' 2015,Syphilis' 2015,Testicular cancer' 2015,Tetanus' 2015,Thalassaemias' 2015,Thyroid cancer' 2015,"Trachea, bronchus, lung cancers' 2015",Trachoma' 2015,Trichomoniasis' 2015,Trichuriasis' 2015,Tuberculosis' 2015,Uncorrected refractive errors' 2015,Unintentional injuries' 2015,Upper respiratory infections' 

In [11]:
wb_who_merge.shape

(157, 2371)

In [12]:
wb_who_merge=wb_who_merge.merge(countries_class[['code','Region','Income group']], on='code', how='left')
wb_who_merge.head(2)

,long,lat,code,country,ARI treatment (% of children under 5 taken to a health provider)' 2007,ARI treatment (% of children under 5 taken to a health provider)' 2008,ARI treatment (% of children under 5 taken to a health provider)' 2009,ARI treatment (% of children under 5 taken to a health provider)' 2010,ARI treatment (% of children under 5 taken to a health provider)' 2011,ARI treatment (% of children under 5 taken to a health provider)' 2012,ARI treatment (% of children under 5 taken to a health provider)' 2013,ARI treatment (% of children under 5 taken to a health provider)' 2014,ARI treatment (% of children under 5 taken to a health provider)' 2015,Birth rate crude (per 1000 people)' 2007,Birth rate crude (per 1000 people)' 2008,Birth rate crude (per 1000 people)' 2009,Birth rate crude (per 1000 people)' 2010,Birth rate crude (per 1000 people)' 2011,Birth rate crude (per 1000 people)' 2012,Birth rate crude (per 1000 people)' 2013,Birth rate crude (per 1000 people)' 2014,Birth rate crude (per 1000 people)' 2015,Births attended by skilled health staff (% of total)' 2007,Births attended by skilled health staff (% of total)' 2008,Births attended by skilled health staff (% of total)' 2009,Births attended by skilled health staff (% of total)' 2010,Births attended by skilled health staff (% of total)' 2011,Births attended by skilled health staff (% of total)' 2012,Births attended by skilled health staff (% of total)' 2013,Births attended by skilled health staff (% of total)' 2014,Births attended by skilled health staff (% of total)' 2015,Depth of the food deficit (kilocalories per person per day)' 2007,Depth of the food deficit (kilocalories per person per day)' 2008,Depth of the food deficit (kilocalories per person per day)' 2009,Depth of the food deficit (kilocalories per person per day)' 2010,Depth of the food deficit (kilocalories per person per day)' 2011,Depth of the food deficit (kilocalories per person per day)' 2012,Depth of the food deficit (kilocalories per person per day)' 2013,Depth of the food deficit (kilocalories per person per day)' 2014,Depth of the food deficit (kilocalories per person per day)' 2015,External resources for health (% of total expenditure on health)' 2007,External resources for health (% of total expenditure on health)' 2008,External resources for health (% of total expenditure on health)' 2009,External resources for health (% of total expenditure on health)' 2010,External resources for health (% of total expenditure on health)' 2011,External resources for health (% of total expenditure on health)' 2012,External resources for health (% of total expenditure on health)' 2013,External resources for health (% of total expenditure on health)' 2014,External resources for health (% of total expenditure on health)' 2015,Fertility rate total (births per woman)' 2007,...,Other urinary diseases' 2015,Other vision loss' 2015,Otitis media' 2015,Ovary cancer' 2015,Pancreas cancer' 2015,Pancreatitis' 2015,Paralytic ileus and intestinal obstruction' 2015,Parasitic and vector diseases' 2015,Parkinson disease' 2015,Peptic ulcer disease' 2015,Periodontal disease' 2015,Poisonings' 2015,Preterm birth complications' 2015,Prostate cancer' 2015,Protein-energy malnutrition' 2015,Rabies' 2015,Respiratory diseases' 2015,Respiratory infections ' 2015,Rheumatic heart disease' 2015,Rheumatoid arthritis' 2015,Road injury' 2015,STDs excluding HIV' 2015,Schistosomiasis' 2015,Schizophrenia' 2015,Self-harm' 2015,Sense organ diseases' 2015,Sickle cell disorders and trait' 2015,Skin diseases' 2015,Stomach cancer' 2015,Stroke' 2015,Sudden infant death syndrome' 2015,Syphilis' 2015,Testicular cancer' 2015,Tetanus' 2015,Thalassaemias' 2015,Thyroid cancer' 2015,"Trachea, bronchus, lung cancers' 2015",Trachoma' 2015,Trichomoniasis' 2015,Trichuriasis' 2015,Tuberculosis' 2015,Uncorrected refractive errors' 2015,Unintentional injuries' 2015,Upper respiratory infections' 2015,Urolithiasis' 2015,Vitamin A deficiency' 2015,Whooping cough' 20

In [13]:
wb_who_merge.shape

(157, 2373)

In [14]:
wb_who_merge.to_pickle('datasets/wb_who_merge.pkl')
wb_who_merge.to_csv('datasets/wb_who_merge.csv', index=False)

In [15]:
# convert df into array, same as Andriy's format in final_wb_array.txt
wb_who_array = wb_who_merge.values
wb_who_array=np.append(wb_who_array, [list(wb_who_merge.columns)],0)

# save to txt
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
with open('datasets/wb_who_merge.txt', 'w') as f:
    f.write(np.array2string(wb_who_array))#, separator=', '))